In [1]:
import pandas as pd

In [8]:
def top_percentuale(df, perc):
    length = df.shape[0]
    end = round(length * perc)
    return df[:end]

In [12]:
path = "./Df completi"
# infl_a = pd.read_csv(f"{path}/influencers_asthma.csv")
infl_a = pd.read_csv(f"{path}/influencers_lungs.csv")
top_infl_a = list(top_percentuale(infl_a, 0.01).Id)

In [13]:
# df_a = pd.read_csv("Df completi/fix/df_asthma_fix.csv")
df_a = pd.read_csv("Df completi/fix/df_lungs_fix.csv")

In [14]:
counter = 0
post = False
old_post = ""
for index, row in df_a.iterrows():
    if row.id.startswith("p"):
        counter += 1
        df_a.loc[index, "thread"] = counter
    else:
        df_a.loc[index, "thread"] = counter

In [15]:
df_a["only_positive"] = True
for thread in df_a["thread"].unique():
    if 2 in df_a[df_a["thread"] == thread].sentiment.unique():
        indexes = df_a[df_a["thread"] == thread].index
        df_a.loc[indexes, "only_positive"] = False

In [4]:
df_a.shape

(132257, 16)

In [5]:
df_l.shape

(595164, 13)

In [17]:
df_a["change_sentiment"] = False
df_a["n_interactions"] = 0
for thread in df_a.thread.unique():   #scorro i vari thread e seleziono solo gli autori con > 1 commento (in modo da vedere se c'è cambio sentiment)
    # if df_a.iloc[0].id.startswith("p"):
    #     post_author = df_a.iloc[0].author
    # else:
    authors = df_a[df_a.thread == thread].author.value_counts().reset_index() 
    list_authors = authors[(authors["count"] > 1)].author.unique()

    if list_authors.shape == (0,):   #se non c'è nessun autore con risposte > 1 si passsa ad altro thread
        continue
    
    for author in list_authors:   # controllo se tra il primo e ultimo commento dell'user c'è commento di un SUPERUSER
        if df_a[(df_a.author == author) & (df_a.sentiment == 0) & (df_a.thread == thread)].shape[0] == 0:
            # n_only_positive += 1
            continue
        indexes_comments = df_a[(df_a.thread == thread) & (df_a.author == author)].index
        min_index = df_a[(df_a.author == author) & (df_a.sentiment == 0) & (df_a.thread == thread)].iloc[0].name
        max_index = max(indexes_comments)
        mid_indexes = list(range(min_index + 1, max_index))
        mid_authors = df_a.loc[mid_indexes].author.unique()
        presence_author = False
        for mid_author in mid_authors:
            if mid_author == "Hidden":
                continue
            if mid_author != author: 
                presence_author = True
        if not presence_author:
            continue
        elif presence_author:   # se è presente un super user -> 
            n_interactions = 0    
            first = True
            negative = False
            interaction = False
            count_interaction = False
            change_sentiment = False
            mid_indexes.insert(0, min_index)  # indexes_comments[0]
            mid_indexes.append(indexes_comments[-1])
            for index in mid_indexes:          # scorro tra i commenti dell'autore in questione
                if (df_a.loc[index].sentiment == 2) and (df_a.loc[index].author == author):  # se trovo commento negativo creo un flag
                    negative = True
                    count_interaction = True
                if first:
                    first = False
                    continue
                if df_a.loc[index].author != author:
                    interaction = True
                    if count_interaction:
                        n_interactions += 1
                        count_interaction = False
                elif (negative) and (df_a.loc[index].sentiment == 2) and (df_a.loc[index].author == author) and interaction: #se c'è cambio di sentiment metto "true" in colonna tecnica
                    change_sentiment = True 
                    ch_sentimant_idx = mid_indexes[: mid_indexes.index(index) + 1]
                    df_a.loc[ch_sentimant_idx, "change_sentiment"] = True
                    df_a.loc[ch_sentimant_idx, "n_interactions"] = n_interactions
                    break
                if  (count_interaction == False) and (df_a.loc[index].author == author):
                    count_interaction = True 
                print(f"sono al thread n. {thread}", end = "\r")

In [18]:
# df_a.to_csv("bozze_df_analysis_sentiment/df_lungs_negative_to_positive.csv", index = False)

In [32]:
# df_a[df_a["change_sentiment"] == True].to_csv("positivo a negativo.csv")

In [2]:
df_a = pd.read_csv("bozze_df_analysis_sentiment/df_lungs_negative_to_positive.csv")  

/tmp/ipykernel_1509/2553562760.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a = pd.read_csv("bozze_df_analysis_sentiment/df_lungs_negative_to_positive.csv")  #mi sono fermato qui


In [3]:
total_threads = set(df_a.thread.unique())
change_sentiment = set(df_a[(df_a["change_sentiment"] == True)].thread.unique())
only_positive = set(df_a[(df_a["only_positive"] == True)].thread.unique())
list_threads = list((total_threads - change_sentiment) - only_positive)

In [4]:
df_a["type_interaction"] = ""     #manca thread solo positivo come caso
for thread in list_threads:  #list_threads: #list_threads:        #list_threads[:100]:  [27]:             #list_threads:
    print(f"thread n. {thread}", end = "\r")
    cases = {}

    authors = df_a[df_a.thread == thread].author.value_counts().reset_index() 
    list_authors = authors.author.unique()   #(authors["count"] > 1) & 
    
    # if list_authors.shape == (0,):   #se non c'è nessun autore con risposte > 1 si passsa ad altro thread
    #     if authors.shape[0] != 0:
    #         thread_index = df_a[df_a.thread == thread].index
    #         df_a.loc[thread_index, "type_interaction"] = "only_su"
    #         continue

    for author in list_authors:

        if df_a[(df_a.author == author) & (df_a.sentiment == 2) & (df_a.thread == thread)].shape[0] == 0:
            continue   #saranno corretti dopo col "only_positive"

        indexes_comments = df_a[(df_a.thread == thread)].index
        min_index = df_a[(df_a.author == author) & (df_a.sentiment == 2) & (df_a.thread == thread)].iloc[0].name
        max_index = max(indexes_comments)
        if min_index == max_index:  #the first negative comment is also the last in the thread
            cases["no_interaction"] = True
            continue
        mid_indexes = list(range(min_index + 1, max_index))
        mid_authors = df_a.loc[mid_indexes].author.unique()
        mid_indexes.append(indexes_comments[-1])
        interaction = False
        change_sentiment = False
        reply_to_su = False
        for index in mid_indexes:
            if df_a.loc[index].author != author:
                interaction = True
            # elif interaction and df_a.loc[index].sentiment == 0 and df_a.loc[index].author == author:
            #     change_sentiment = True
            #     ch_sentimant_idx = mid_indexes[: mid_indexes.index(index) + 1]
            #     df_a.loc[ch_sentimant_idx, "type_interaction"] = "change_sentiment"
            elif interaction and df_a.loc[index].author == author:
                reply_to_su = True
        if not interaction:
            cases["no_interaction"] = True
        elif reply_to_su:
            cases["no_change"] = True
        elif not reply_to_su:
            cases["no_reply_to_su"] = True
        if "no_change" in cases.keys():
            thread_index = df_a[df_a.thread == thread].index
            df_a.loc[thread_index, "type_interaction"] = "no_change"
    if "no_change" in cases.keys():
        thread_index = df_a[df_a.thread == thread].index
        df_a.loc[thread_index, "type_interaction"] = "no_change"
        continue
    if "no_reply_to_su" in cases.keys():
        thread_index = df_a[df_a.thread == thread].index
        df_a.loc[thread_index, "type_interaction"] = "no_reply_to_other_user"
        continue
    if "no_interaction" in cases.keys():
        thread_index = df_a[df_a.thread == thread].index
        df_a.loc[thread_index, "type_interaction"] = "no_interaction"
        continue

    # if n_only_positive == len(list_authors): #non tengo in conto dei superusers
    #     thread_index = df_a[df_a.thread == thread].index
    #     df_a.loc[thread_index, "type_interaction"] = "only_positive"
    #     continue

    thread_index = df_a[df_a.thread == thread].index
    df_a.loc[thread_index, "type_interaction"] = "caso_estremo"

In [5]:
change_sentiment_index = df_a[df_a.thread.isin(df_a[df_a.change_sentiment == True].thread.unique())].index
only_positive_index = df_a[df_a.thread.isin(df_a[df_a.only_positive == True].thread.unique())].index

df_a.loc[change_sentiment_index, "type_interaction"] = "change_sentiment"
df_a.loc[only_positive_index, "type_interaction"] = "only_positive"

In [13]:
df_a["is_su"] = df_a.author.apply(lambda x: True if x in top_infl_a else False)

In [14]:
df_a["category_interaction"] = ""
for thread in df_a[df_a.change_sentiment == True].thread.unique():
    df = df_a[(df_a.thread == thread) & (df_a.change_sentiment == True)]
    authors_and_type = df[["author", "is_su"]]
    author_changed_sentiment = df.iloc[0].author
    if authors_and_type[authors_and_type.author != author_changed_sentiment]["is_su"].unique().shape[0] == 1:  #se c'è solo su = True
        indexes = df_a[df_a.thread == thread].index
        df_a.loc[indexes, "category_interaction"] = "only_su"
    else:  # se c'è sia is_su = true e is_su = false
        indexes = df_a[df_a.thread == thread].index
        df_a.loc[indexes, "category_interaction"] = "su_&_user"
    print(f"thread n. {thread}", end = "\r")

In [35]:
# df_a.to_csv("bozze_df_analysis_sentiment/df_lungs_negative_to_positive.csv", index=False)

In [23]:
df_a = pd.read_csv("bozze_df_analysis_sentiment/df_lungs_negative_to_positive.csv")

In [30]:
for index in df_a["thread"].unique():
    authors = df_a[df_a["thread"] == index]["author"].unique()
    for author in authors:
        indexes = df_a[df_a["thread"] == index].index
        if author in top_infl_a:
            # print(author, index)
            df_a.loc[indexes, "su_in_thread"] = True
            break
    print(index, end = "\r")

### BLF

In [32]:
df_a = df_a[(df_a["su_in_thread"] == True)]

In [33]:
df_stats_type = df_a.groupby("thread")["type_interaction"].unique().reset_index()
df_stats_type["type_interaction"] = df_stats_type["type_interaction"].apply(lambda x: x[0]) 
df_stats_type.value_counts("type_interaction")

type_interaction
only_positive             38760
no_reply_to_other_user    22740
no_change                 14271
no_interaction             5274
change_sentiment           3010
Name: count, dtype: int64

In [36]:
df_category_type = df_a[df_a.change_sentiment == True].groupby("thread")["category_interaction"].unique().reset_index()
df_category_type["category_interaction"] = df_category_type["category_interaction"].apply(lambda x: x[0]) 
df_category_type.value_counts("category_interaction")

category_interaction
only_su      2468
su_&_user     542
Name: count, dtype: int64

### Asthma UK

In [37]:
df_a = pd.read_csv("bozze_df_analysis_sentiment/df_asthma_negative_to_positive.csv")
df_a = df_a[(df_a["su_in_thread"] == True)]

In [38]:
change_sentiment_index = df_a[df_a.thread.isin(df_a[df_a.change_sentiment == True].thread.unique())].index
only_positive_index = df_a[df_a.thread.isin(df_a[df_a.only_positive == True].thread.unique())].index

df_a.loc[change_sentiment_index, "type_interaction"] = "change_sentiment"
df_a.loc[only_positive_index, "type_interaction"] = "only_positive"

In [39]:
df_stats_type = df_a.groupby("thread")["type_interaction"].unique().reset_index()
df_stats_type["type_interaction"] = df_stats_type["type_interaction"].apply(lambda x: x[0]) 
df_stats_type.value_counts("type_interaction")

type_interaction
no_reply_to_other_user    4486
only_positive             3872
no_change                 3512
no_interaction            1471
change_sentiment           715
Name: count, dtype: int64